1. Make PMJAY Process Flow Cypher Script for Knowledge Graph
2. Create a dictionary of properties for each entity so that it can be used to get appropriate data for each process flow of a patient

In [1]:
#Mounting Google Drive

from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


Get entities from the Process Flow Sentences

In [21]:
import pandas as pd
pfs= pd.read_csv('/content/drive/MyDrive/Final (Completed)/process_flow_sentences_2.csv')

In [22]:
sentences= pfs['sentences']

In [23]:
import spacy
pmjay_nlp= spacy.load('/content/drive/MyDrive/Final (Completed)/nlp_model_2')

/usr/local/lib/python3.9/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


Extracting entities from Process Flow Sentences

In [24]:
#Extration of Entities from pfs sentences also create ent-label dictionary
#v2

ent_pair=[] # [(sentence,[ent_1, ent_2]),...]
el_dict={} #Entity-Label dictionary
for i in range(0,len(sentences)):
    e_pair=[]
    d= pmjay_nlp(sentences[i])
    for ent in d.ents:        
        e_pair.append(ent)
        el_dict[ent.text]=ent.label_
    if(len(e_pair)>1):
        ent_pair.append((sentences[i],e_pair))
print("Entity Pairs:",len(ent_pair),"\n",ent_pair)
print("Entity Label Dict:",len(el_dict),"\n",el_dict)

Entity Pairs: 94 
 [('The Patient checks eligibility through Call-Centre', [Patient, Call-Centre]), ('The Patient checks eligibility through PMJAY Kiosk', [Patient, PMJAY Kiosk]), ('The Patient checks eligibility through EHCP Registration Desk', [Patient, EHCP Registration Desk]), ('PMJAY Kiosk is assigned a PMAM', [PMJAY Kiosk, PMAM]), ('EHCP Registration Desk is assigned a PMAM', [EHCP Registration Desk, PMAM]), ('PMAM verifies identity and eligibility through BIS', [PMAM, BIS]), ('BIS creates Silver Record', [BIS, Silver Record]), ('BIS creates Golden Record', [BIS, Golden Record]), ('BIS creates e-card', [BIS, e-card]), ('e-card is given to the PMJAY Beneficiary', [e-card, PMJAY Beneficiary]), ('PMAM informs about entitlements to the PMJAY Beneficiary', [PMAM, PMJAY Beneficiary]), ('Patient becomes PMJAY Beneficiary', [Patient, PMJAY Beneficiary]), ('PMJAY Beneficiary is sent for Consultation', [PMJAY Beneficiary, Consultation]), ('Consultation is done by General Doctor', [Consulta

In [25]:
#Adding Missing Data
el_dict['Am I Eligible App']='MOBILE_APP'
el_dict['Am I Eligible Website']='PORTAL'
el_dict['Medication']='PROCESS'
el_dict['Hospitalization']='PROCESS'
ent_pair.append((("The Patient checks eligibility through Am I Eligible App"),['Patient','Am I Eligible App']))
ent_pair.append((("The Patient checks eligibility through Am I Eligible Website"),['Patient','Am I Eligible Website']))
ent_pair.append((("General Doctor decides Hospitalization"),['General Doctor','Hospitalization']))
ent_pair.append((("General Specialist decides Hospitalization"),['General Specialist','Hospitalization']))
ent_pair.append((("General Doctor decides Medication"),['General Doctor','Medication']))
ent_pair.append((("General Specialist decides Medication"),['General Specialist','Medication']))
print("Entity Pairs:",len(ent_pair),"\n",ent_pair)
print("Entity Label Dict:",len(el_dict),"\n",el_dict)

Entity Pairs: 100 
 [('The Patient checks eligibility through Call-Centre', [Patient, Call-Centre]), ('The Patient checks eligibility through PMJAY Kiosk', [Patient, PMJAY Kiosk]), ('The Patient checks eligibility through EHCP Registration Desk', [Patient, EHCP Registration Desk]), ('PMJAY Kiosk is assigned a PMAM', [PMJAY Kiosk, PMAM]), ('EHCP Registration Desk is assigned a PMAM', [EHCP Registration Desk, PMAM]), ('PMAM verifies identity and eligibility through BIS', [PMAM, BIS]), ('BIS creates Silver Record', [BIS, Silver Record]), ('BIS creates Golden Record', [BIS, Golden Record]), ('BIS creates e-card', [BIS, e-card]), ('e-card is given to the PMJAY Beneficiary', [e-card, PMJAY Beneficiary]), ('PMAM informs about entitlements to the PMJAY Beneficiary', [PMAM, PMJAY Beneficiary]), ('Patient becomes PMJAY Beneficiary', [Patient, PMJAY Beneficiary]), ('PMJAY Beneficiary is sent for Consultation', [PMJAY Beneficiary, Consultation]), ('Consultation is done by General Doctor', [Consult

In [26]:
el_df= pd.DataFrame(el_dict, index=[0])
entities=pd.DataFrame(el_df.columns)

In [27]:
entities.sample(3)

,0
9,PMJAY Beneficiary
37,Am I Eligible Website
35,Doctor


Output: **`entities`**

Getting Relations from the Process Flow Sentences

In [28]:
#Extraction of Relations and creating Level 1 triplets for KG
import re
triplets_1=[] # [[ent_1, ent_2, relation],...]
for e in ent_pair:
    e1= str(e[1][0])
    e2= str(e[1][1])
    #if(e1!='Doctor' and e2!='Doctor'):
        #sent= e[0]
        #result= re.search(e1+"(.*)"+e2,sent)
        #triplets_1.append([e1,e2,result.group(1).strip()])
    sent= e[0]
    result= re.search(e1+"(.*)"+e2,sent)
    triplets_1.append([e1,e2,result.group(1).strip()])
triplets_1

[['Patient', 'Call-Centre', 'checks eligibility through'],
 ['Patient', 'PMJAY Kiosk', 'checks eligibility through'],
 ['Patient', 'EHCP Registration Desk', 'checks eligibility through'],
 ['PMJAY Kiosk', 'PMAM', 'is assigned a'],
 ['EHCP Registration Desk', 'PMAM', 'is assigned a'],
 ['PMAM', 'BIS', 'verifies identity and eligibility through'],
 ['BIS', 'Silver Record', 'creates'],
 ['BIS', 'Golden Record', 'creates'],
 ['BIS', 'e-card', 'creates'],
 ['e-card', 'PMJAY Beneficiary', 'is given to the'],
 ['PMAM', 'PMJAY Beneficiary', 'informs about entitlements to the'],
 ['Patient', 'PMJAY Beneficiary', 'becomes'],
 ['PMJAY Beneficiary', 'Consultation', 'is sent for'],
 ['Consultation', 'General Doctor', 'is done by'],
 ['Consultation', 'General Specialist', 'is done by'],
 ['PMJAY Beneficiary', 'General Doctor', 'consults a'],
 ['PMJAY Beneficiary', 'General Specialist', 'consults a'],
 ['General Doctor', 'Diagnostics', 'decides'],
 ['General Doctor', 'Pre-Authorization Form', 'fills'

Output: **`triplets_1 =`** <br>
 `[['Patient', 'Am I Eligible App', 'checks eligibility through'],`<br>`
 ['Patient', 'Am I Eligible Website', 'checks eligibility through'],`<br>`
 ['Patient', 'Call-Centre', 'checks eligibility through'],`<br>`
 ['Patient', 'PMJAY Kiosk', 'checks eligibility through'],`<br>`
 ['Patient', 'EHCP Registration Desk', 'checks eligibility through'],`<br>`...,`<br>`
 ['PMJAY Kiosk', 'PMAM', 'is assigned a']]`

Assigning Variables to each Entity

In [29]:
#Assigning Variables to each Entity
#Method to generate variables
def generate_variable(n):
    alphabet = "abcdefghijklmnopqrstuvwxyz"
    sequences = []
    if n < 26:
        return(alphabet[n])
    elif n < 702:
        first = alphabet[(n // 26) - 1]
        second = alphabet[n % 26]
        return(first + second)
    else: 
        first = alphabet[(((n//26)-1)//26)-1]
        second = alphabet[(((n//26)-1)%26)]
        third = alphabet[n%26]
        return(first+second+third)
    return sequences

ev_dict={}
var=[]
for i in range(0,len(entities)):
    var=generate_variable(i)
    e=str(entities[0][i])
    ev_dict[e]=var
ev_dict   

{'Patient': 'a',
 'Call-Centre': 'b',
 'PMJAY Kiosk': 'c',
 'EHCP Registration Desk': 'd',
 'PMAM': 'e',
 'BIS': 'f',
 'Silver Record': 'g',
 'Golden Record': 'h',
 'e-card': 'i',
 'PMJAY Beneficiary': 'j',
 'Consultation': 'k',
 'General Doctor': 'l',
 'General Specialist': 'm',
 'Diagnostics': 'n',
 'Pre-Authorization Form': 'o',
 'MEDCO': 'p',
 'Empanelled Hospital': 'q',
 'Pre-Authorization Request': 'r',
 'ISA': 's',
 'SHA': 't',
 'Insurance Company': 'u',
 'TMS': 'v',
 'Picture': 'w',
 'Treatment': 'x',
 'Treating Doctor': 'y',
 'Treating Specialist': 'z',
 'Treatment Report': 'aa',
 'Medical Package': 'ab',
 'PMJAY': 'ac',
 'Treatment Procedure': 'ad',
 'Claim': 'ae',
 'NHA': 'af',
 'Medical Report': 'ag',
 'EHCP': 'ah',
 'Feedback': 'ai',
 'Doctor': 'aj',
 'Am I Eligible App': 'ak',
 'Am I Eligible Website': 'al',
 'Medication': 'am',
 'Hospitalization': 'an'}

Preparing a dictionary of entities and properties

In [30]:
ent_prop= pd.read_csv('/content/drive/MyDrive/Final (Completed)/ent_properties.csv')
ent_prop

,entities,properties
0,Patient,patient_id
1,Patient,name
2,Patient,gender
3,Patient,age
4,Patient,address
...,...,...
68,Picture,dp_taken
69,Picture,ap_id
70,Picture,ap_taken
71,PMJAY,scheme_name


In [31]:
ent_prop_dict={} #A dictionary to store properties of each entity
#Now make ent_properties dictionary from ent_prop dataframe with entity as the key
for index,row in ent_prop.iterrows():
    entity= row['entities']
    prop=row['properties']
    if (entity not in ent_prop_dict):
        ent_prop_dict[entity]=[prop]
    else:
        ent_prop_dict[entity].append(prop)

In [ ]:
print(ent_prop_dict)
print(ent_prop_dict['Patient'])
print(ent_prop_dict['Patient'][0])
print(ent_prop_dict['Patient'][3])

{'Patient': ['patient_id', 'name', 'gender', 'age', 'address', 'father_husband_name', 'mobile_no', 'aadhar_no', 'ration_card_no'], 'Am I Eligible App': ['app_acc_id'], 'Am I Eligible Website': ['website_acc_id', 'url'], 'Call-Centre': ['caller_id', 'call_centre_no'], 'PMJAY Kiosk': ['kiosk_id', 'location'], 'EHCP Registration Desk': ['ehcp_name'], 'PMAM': ['pmam_id', 'pmam_name'], 'BIS': ['socio_economic_profiling', 'priority_household', 'exclusion_criteria', 'verification'], 'Silver Record': ['sr_id'], 'Golden Record': ['gr_id'], 'e-card': ['ecard_no', 'pmjay_id', 'ehcp_name', 'validity'], 'PMJAY Beneficiary': ['ecard_no', 'admitted'], 'Consultation': ['consultation_no'], 'General Doctor': ['gd_id', 'gd_name'], 'General Specialist': ['gs_id', 'gs_name'], 'Hospitalization': ['required'], 'Diagnostics': ['required'], 'Medication': ['required'], 'Pre-Authorization Form': ['form_no'], 'Empanelled Hospital': ['ehcp_id'], 'MEDCO': ['medco_id'], 'Pre-Authorization Request': ['pre_ar_id'], 'S

Output: **`ent_prop_dict`**<br>`{'Patient': ['patient_id', 'name', 'gender', 'age', 'address', 'father_husband_name', 'mobile_no', 'aadhar_no', 'ration_card_no']`

PMJAY Dummy Records

In [32]:
#Get the Document
import pandas as pd
pdr= pd.read_csv('/content/drive/MyDrive/Final (Completed)/pmjay_dummy_record.csv')
print(pdr.columns)

Index(['patient_id', 'name', 'gender', 'age', 'address', 'father_husband_name',
       'mobile_no', 'aadhar_no', 'ration_card_no', 'app_acc_id',
       'website_acc_id', 'url', 'caller_id', 'call_centre_no', 'kiosk_id',
       'location', 'ehcp_name', 'pmam_id', 'pmam_name',
       'socio_economic_profiling', 'priority_household', 'exclusion_criteria',
       'verification', 'sr_id', 'gr_id', 'ecard_no', 'pmjay_id', 'ehcp_name.1',
       'validity', 'ecard_no.1', 'admitted', 'consultation_no', 'gd_id',
       'gd_name', 'gs_id', 'gs_name', 'required', 'required.1', 'required.2',
       'form_no', 'ehcp_id', 'medco_id', 'pre_ar_id', 'sha_id', 'state',
       'nha_name', 'ic_id', 'ic_name', 'ap_id', 'ap_taken', 'tms_rec_id',
       't_id', 't_name', 'tp_id', 'tp_name', 'td_id', 'td_name', 'ts_id',
       'ts_name', 'tr_id', 'tp_id.1', 'tp1_taken', 'tp2_taken', 'mp_id',
       'mp_name', 'c_id', 'mr_id', 'dp_id', 'dp_taken', 'f_id', 'isa_id',
       'scheme_name', 'full_form'],
      dtyp

In [ ]:
value=pdr['app_acc_id'][0]
value_str= str(value)
print(value,type(value), isinstance(value,float), value_str, type(value_str))

1.0 <class 'numpy.float64'> True 1.0 <class 'str'>


In [33]:
#Data Cleaning: Convertin Nan Values to "n/a" and float type into integer type
for index, row in pdr.iterrows():
    for col, value in row.iteritems():
        # check if value is NaN
        if isinstance(value,float):
            # convert NaN to "n/a"
            if str(value)=='nan':
                pdr.at[index, col] = "n/a" 
            # convert float to integer
            else:
                pdr.at[index, col] = int(value)

For each record of `pdr`, build a process flow knowledge graph

In [34]:
#Starting with a single record
rec= pdr.iloc[0]
query_chunk=[]
c_query=[]
for i in range(0,len(pdr)):
    rec= pdr.iloc[i]
    #Add values to the properties of each entity for each record
    ent_prop_dict #Entity-Properties Names Dictionary
    pv_dict={} #Property-Values Dictionary
    ep_dict={} #Entity-Properties with values Dictionary 
    for e in entities[0]:
        if e != 'Doctor':
            for p in ent_prop_dict[e]:
                pv_dict[p]=rec[p]
            ep_dict[e]=pv_dict.copy()
        pv_dict.clear()
    #Converting the dictionary into required cypher format to create cypher query to create nodes
    for key,value in ep_dict.items():
        properties=""
        for key2,val2 in value.items():
            if isinstance(val2,float):
                val2= int(val2)
            properties+= f"{key2}:\"{val2}\","
        properties+=f"type:\"{key}\""
        #properties= properties[:-1]
        c_query.append("CREATE ("+ev_dict[key]+":"+el_dict[key]+"{"+properties+"})")
    #c_query
    #Use rel_pair to make relations cypher query
    rel_pairs=[] #(source entity's variable, relation, target entitiy's variable)
    for r in triplets_1:
        rel_pairs.append((ev_dict[r[0]],r[2],ev_dict[r[1]]))

    #Cypher Query for Creating Relations between entities
    # CREATE (a)-[:relation]->(b)
    for r in rel_pairs:
        rel= re.sub(' ','_',r[1])
        c_query.append("CREATE ("+r[0]+")-[:"+rel+"]->("+r[2]+")")

    cypher_text=""
    for q in c_query:
        cypher_text=cypher_text+q+"\n"
    cypher_text=cypher_text.strip()    

    query_chunk.append(cypher_text)
    c_query.clear()



In [35]:
print(query_chunk)

['CREATE (a:PERSON{patient_id:"1",name:"N1",gender:"male",age:"63",address:"Bijanbari",father_husband_name:"HF1",mobile_no:"9876543210",aadhar_no:"1234-1234-1234",ration_card_no:"2345-2345-2345",type:"Patient"})\nCREATE (b:ORG{caller_id:"n/a",call_centre_no:"n/a",type:"Call-Centre"})\nCREATE (c:CSC{kiosk_id:"n/a",location:"n/a",type:"PMJAY Kiosk"})\nCREATE (d:CSC{ehcp_name:"ECHP1",type:"EHCP Registration Desk"})\nCREATE (e:PERSON{pmam_id:"1",pmam_name:"PMAM1",type:"PMAM"})\nCREATE (f:PORTAL{socio_economic_profiling:"yes",priority_household:"yes",exclusion_criteria:"no",verification:"yes",type:"BIS"})\nCREATE (g:DOCUMENT{sr_id:"1",type:"Silver Record"})\nCREATE (h:DOCUMENT{gr_id:"1",type:"Golden Record"})\nCREATE (i:DOCUMENT{ecard_no:"1",pmjay_id:"1",ehcp_name:"ECHP1",validity:"05-02-2035",type:"e-card"})\nCREATE (j:PERSON{ecard_no:"1",admitted:"yes",type:"PMJAY Beneficiary"})\nCREATE (k:PROCESS{consultation_no:"1",type:"Consultation"})\nCREATE (l:PERSON{gd_id:"1",gd_name:"GDN1",type:"G

In [36]:
print(type(query_chunk))
query_df= pd.DataFrame(query_chunk)
query_df.to_csv("/content/drive/MyDrive/Final (Completed)/pmjay_dummy_cypher.csv",index=False)

<class 'list'>


### Conclusion:
The `pmjay_dummy_cypher.csv` contains the cypher query for each record of a patient. And since all the query for all the patient records could not be run simultaneously, the query needs to be given one ata a time. <br>
Given Google Colab runtime is Google Cloud, it cannot connect with Neo4j Desktop which runs on local IP, hence we move to anaconda jupyter environment whose runtime in the local system which matches with Neo4j Desktop runtime.